### 시퀀스를 결과로 만들어내는 함수를 만들 때
### 1) 리스트를 반환
#### 문제점1: 코드에 잡음이 많고 핵심을 알아보기 어렵다
- 리스트 메소드 호출들로 인해 리스트에 추가될 값 (index + 1)의 중요성이 희석된다.

#### 문제점2: 함수를 반환하기 전에 리스트에 모든 결과를 저장해야 한다
- 입력이 매우 크면 프로그램이 메모리를 소진해서 중단될 수 있다.

In [1]:
def index_words(text):
    result = []
    # text가 ''가 아니라면
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        # 공백의 인덱스+1을 리스트에 추가
        if letter == ' ':
            result.append(index + 1)
    return result

In [2]:
address = '컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어:전산기)는 진공관' # 186쪽 오타 정정
result = index_words(address)
print(result[:10])

[0, 8, 18, 23, 28, 38]


### 2) 제너레이터를 사용
- 함수 안에서 yield식을 사용하면 함수는 제너레이터가 된다.
- next 내장함수를 호출할 때마다 이터레이터는 제너레이터 함수를 다음 yield식까지 진행시킨다. 

> **이터레이터(iterator)**는 값을 차례대로 꺼낼 수 있는 객체(object)입니다. 파이썬에서는 이터레이터만 생성하고 <u>값이 필요한 시점이 되었을 때 값을 만드는 방식</u>을 사용합니다. 즉, 데이터 생성을 뒤로 미루는 것인데 이런 방식을 **지연 평가(lazy evaluation)**라고 합니다.

![img](./bt30.PNG)

(이미지출처: [Unit 40. 제너레이터 사용하기](https://dojang.io/mod/page/view.php?id=2412))

#### 문제점1 해결: 읽기 쉬워진다.

In [3]:
def index_words_iter(text):
    if text:
        print('전')
        yield 0
        print('후')
    for index, letter in enumerate(text):
        if letter == ' ':
            print('전')
            yield index + 1
            print('후')

In [4]:
it = index_words_iter(address)
it # 제너레이터 객체

<generator object index_words_iter at 0x000001F312A5C120>

 #### dir 함수를 사용하면 객체의 메서드를 확인할 수 있다.
 이터레이터에서 볼 수 있는 \__iter\__, \__next\__ 메서드가 들어있다.

In [5]:
dir(it)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [6]:
print(next(it))

전
0


### 이터레이터를 쉽게 리스트로 변환 가능

In [7]:
result = list(index_words_iter(address))
print(result[:10])

전
후
전
후
전
후
전
후
전
후
전
후
[0, 8, 18, 23, 28, 38]


#### 문제점2 해결: 사용하는 메모리크기를 어느정도 제한할 수 있다.
- 한 번의 한 줄씩 읽음: 함수의 작업 메모리는 입력 중 가장 긴 줄의 길이로 제한된다.

In [8]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

**itertools의 islice**는 시퀀스의 슬라이싱과 같이 동작한다.<br>
첫 번째 매개 변수는 반복 가능한 객체, 두 번째 매개 변수는 시작 index, 세 번째 매개 변수는 마지막 index+1이다.<br>
stride를 지정하는 매개변수를 마지막에 추가할 수 있다. 

In [9]:
import itertools
with open('address.txt', 'r', encoding='utf-8') as f:
    it = index_file(f)
    results = itertools.islice(it, 0, 10)
    print(list(results))

[0, 8, 18, 23, 28, 38]


itertolls.islice 사용 예시

In [10]:
from itertools import islice
      
li = [2, 4, 5, 7, 8, 10, 20] 

print(list(itertools.islice(li, 1, 6, 2)))  

[4, 7, 10]


#### 제너레이터를 정의할 때 알아둬야 할 점!
**-> 제너레이터가 반환하는 이터레이터에 상태가 있기 때문에 호출하는 쪽에서 재사용이 불가능하다!!**